## 准备工作

#### 1. 用 https://www.kanbilibili.com/ 来下载 B站视频和弹幕
例如：https://www.kanbilibili.com/video/av54944922?from=search&seid=13855581700550241007#download

#### 2. 然后用 DanmakuFactory_1.11_release 来将 xml 文件转化为 ass 字幕文件

#### 3. 使用格式工厂将视频和ass进行合成，产生原视频和字幕视频

#### 4. 下载 https://github.com/matterport/Mask_RCNN 源代码，在sample目录下运行该notebook

#### 5. 修改配置文件

In [1]:
import logging
import os
import sys
from time import time as timer
import cv2
import numpy as np

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
def get_video_shape(originVideo, danmuVideo):
    '''
    检查原视频和弹幕视频的shape是否相同
    :param danmu:
    :param origin:
    :return: shape
    '''
    # 读取一帧的视频
    _, frame_ori = originVideo.read()
    _, frame_dan = danmuVideo.read()
    assert frame_dan.shape == frame_ori.shape, \
        "The height and width are not the same! \n {} with {}".format(frame_ori.shape, frame_dan.shape)
    return frame_dan.shape

In [3]:
def get_video_frame(danmuVideo, originVideo):
    '''
    检查视频帧率
    :param danmu:
    :param origin:
    :return:
    '''
    if originVideo.get(cv2.CAP_PROP_FPS) != danmuVideo.get(cv2.CAP_PROP_FPS):
        logger.warning('frame not same!')
        logger.warning('origin:{}'.format(originVideo.get(cv2.CAP_PROP_FPS)))
        logger.warning('danmu:{}'.format(danmuVideo.get(cv2.CAP_PROP_FPS)))
    return originVideo.get(cv2.CAP_PROP_FPS)

In [4]:
def set_mask(originVideo, danmuVideo, mask):
    '''
    将 原帧的 蒙版区域放到 弹幕视频帧 之上
    :param origin:
    :param danmu:
    :param mask:
    :return:
    '''
    for c in range(3):
        danmuVideo[:, :, c] = np.where(mask == 1,
                                  originVideo[:, :, c],
                                  danmuVideo[:, :, c])
    return danmuVideo

In [5]:
def display_instances(originVideo, danmuVideo, boxes, masks, ids, scores):
    '''
    返回处理蒙版弹幕后的图片
    :param origin: 原始图片
    :param danmu: 带弹幕的图片
    :param boxes: 框, [num_instances, ... ...]
    :param masks: 掩膜，[h, w, num_instances]
    :param ids: 各个mask所属的class id, [num_instances]
    :param scores:  置信度
    :return:
    '''
    n_instances = boxes.shape[0]
    h, w, _ = originVideo.shape

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue
        if ids[i] != 1:
        	# 1：person
            continue
        mask = masks[:, :, i]

        danmuVideo = set_mask(originVideo, danmuVideo, mask)

    return danmuVideo

In [ ]:
def processVideo(model, params):
    '''
    返回人物防遮挡蒙版弹幕视频
    :param model: Mask-RCNN模型
    :param params: 配置dict
    :return:
    '''
    if not os.path.exists(params.get('originVideo')):
        logger.warning('originVideo video is not exists!')
        exit(0)

    if not os.path.exists(params.get('danmuVideo')):
        logger.warning('danmuVideo video is not exists!')
        exit(0)

    # opencv打开视频
    originVideo = cv2.VideoCapture(params.get('originVideo'))
    danmuVideo = cv2.VideoCapture(params.get('danmuVideo'))

    saveVideo = params.get('saveVideo')

    # 检查两个视频的大小是否吻合，返回shape
    height, width, _ = get_video_shape(danmuVideo, originVideo)

    # 检查帧率是否吻合
    fps = get_video_frame(danmuVideo, originVideo)

    if saveVideo:
        if not os.path.exists(params.get('output').split('/')[0]):
            os.makedirs(params.get('output').split('/')[0])
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        fps = round(danmuVideo.get(cv2.CAP_PROP_FPS))
        videoWriter = cv2.VideoWriter(
            params.get('output'), fourcc, fps, (width, height))

    elapsed = int()  # 记录处理了多少帧
    start = timer()
    stop = fps * params.get('keep')

    while originVideo.isOpened():
        elapsed += 1
        _, frame_ori = originVideo.read()
        _, frame_dan = danmuVideo.read()
        if frame_ori is None or frame_dan is None:
            logger.info('\nEnd of Video')
            break
        results = model.detect([frame_ori], verbose=0)
        r = results[0]
        frame = display_instances(frame_ori, frame_dan, r['rois'], r['masks'], r['class_ids'], r['scores'])

        if saveVideo:
            videoWriter.write(frame)

        # 输出fps
        if elapsed % 5 == 0:
            sys.stdout.write('\r')
            sys.stdout.write('{0:3.3f} FPS'.format(
                elapsed / (timer() - start)))
            sys.stdout.flush()

        if elapsed == stop:
            logger.info('Made a video of {} seconds.'.format(params.get('keep')))
            break

    sys.stdout.write('\n')
    if saveVideo:
        videoWriter.release()
    originVideo.release()
    danmuVideo.release()